Задание 1. Проведите анализ окупаемости трафика для стран (RU, IN, MX, AU)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from operator import attrgetter
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
def date_preprocess(s):
    try:
        return datetime.strptime(s+":00", '%Y-%m-%d %H:%M:%S.%f%z')
    except ValueError:
        return datetime.strptime(s+":00", '%Y-%m-%d %H:%M:%S%z')

In [3]:
user_data = pd.read_csv("data/user_data.csv")
user_data['install_time'] = user_data['install_time'].apply(date_preprocess)
user_data['event_time'] = user_data['event_time'].apply(date_preprocess)
user_data.head()

,attributed_touch_time,install_time,event_time,event_name,event_revenue_usd,media_source,channel,campaign_id,adset_id,ad_id,country_code,user_id
0,NaN,2021-01-01 00:08:10.721000+00:00,2021-01-01 00:08:10.721000+00:00,install,NaN,NaN,NaN,NaN,NaN,NaN,VN,1609459686080-7483030925743023347
1,NaN,2021-01-01 00:39:23.948000+00:00,2021-01-01 00:39:23.948000+00:00,install,NaN,NaN,NaN,NaN,NaN,NaN,US,1609461562630-105981105396801528
2,NaN,2021-01-01 00:40:36.525000+00:00,2021-01-01 00:40:36.525000+00:00,reinstall,NaN,NaN,NaN,NaN,NaN,NaN,CZ,1609461625904-7673362380866907786
3,NaN,2021-01-01 00:49:03.486000+00:00,2021-01-01 00:49:03.486000+00:00,reinstall,NaN,NaN,NaN,NaN,NaN,NaN,PT,1609462209340-904240965619919015
4,NaN,2021-01-01 00:51:53.395000+00:00,2021-01-01 00:51:53.395000+00:00,install,NaN,NaN,NaN,NaN,NaN,NaN,US,1609279214347-3174174


In [4]:
ads_data = pd.read_csv("data/ads_data.csv")
ads_data.head()

,date,campaign_id,ad_set_id,ad_id,cpm,ctr,spend,clicks,impressions,breakdown_set,breakdowns,platform
0,2021-01-01,23847157133130369,23847157133120369,23847157133270369,11.592920,0.000000,3.93,0,339,{country},"{""country"": ""UK""}",ios
1,2021-01-01,23847157133170369,23847157133180369,23847157133260369,8.307427,0.518135,4.81,3,579,{country},"{""country"": ""UK""}",android
2,2021-01-01,23847157133130369,23847157133120369,23847157133250369,23.848580,0.630915,7.56,2,317,{country},"{""country"": ""UK""}",ios
3,2021-01-01,23847157133130369,23847157133120369,23847157133240369,17.602996,0.374532,4.70,1,267,{country},"{""country"": ""UK""}",ios
4,2021-01-01,23847157133130369,23847157133120369,23847157133230369,23.512397,0.000000,5.69,0,242,{country},"{""country"": ""UK""}",ios


In [5]:
def str_to_dic(s):
    if s != '{}':
        return eval(s)['country']
    else:
        return ''

In [6]:
ads_data['breakdowns_country'] = ads_data['breakdowns'].apply(str_to_dic)
ads_data.head()

,date,campaign_id,ad_set_id,ad_id,cpm,ctr,spend,clicks,impressions,breakdown_set,breakdowns,platform,breakdowns_country
0,2021-01-01,23847157133130369,23847157133120369,23847157133270369,11.592920,0.000000,3.93,0,339,{country},"{""country"": ""UK""}",ios,UK
1,2021-01-01,23847157133170369,23847157133180369,23847157133260369,8.307427,0.518135,4.81,3,579,{country},"{""country"": ""UK""}",android,UK
2,2021-01-01,23847157133130369,23847157133120369,23847157133250369,23.848580,0.630915,7.56,2,317,{country},"{""country"": ""UK""}",ios,UK
3,2021-01-01,23847157133130369,23847157133120369,23847157133240369,17.602996,0.374532,4.70,1,267,{country},"{""country"": ""UK""}",ios,UK
4,2021-01-01,23847157133130369,23847157133120369,23847157133230369,23.512397,0.000000,5.69,0,242,{country},"{""country"": ""UK""}",ios,UK


In [7]:
user_data.loc[(user_data['event_name'] != "af_purchase") & (user_data['event_name'] != "af_subscribe")].loc[user_data['event_revenue_usd'].notna() & user_data['event_revenue_usd'] != 0]


,attributed_touch_time,install_time,event_time,event_name,event_revenue_usd,media_source,channel,campaign_id,adset_id,ad_id,country_code,user_id


In [10]:
user_data = user_data.loc[(user_data['event_name'] == "af_purchase") | (user_data['event_name'] == "af_subscribe")]

In [37]:
analysis_data = pd.DataFrame(user_data.groupby('country_code').agg(revenue=('event_revenue_usd','sum')))
analysis_data['spend'] = pd.DataFrame(ads_data.groupby('breakdowns_country')['spend'].agg(sum='sum'))

In [38]:
analysis_data = analysis_data.loc[analysis_data.index.isin(['RU', 'IN', 'MX', 'AU'])]

In [40]:
analysis_data['ROI'] = (analysis_data['revenue'] - analysis_data['spend'])/analysis_data['spend']*100

In [41]:
analysis_data['ROAS'] = analysis_data['revenue']/analysis_data['spend']*100

In [42]:
analysis_data

,revenue,spend,ROI,ROAS
country_code,,,,
AU,80339.778822,102771.106086,-21.826492,78.173508
IN,13025.643714,12023.997382,8.330394,108.330394
MX,7716.555016,24891.971021,-68.999823,31.000177
RU,62357.618019,52203.413104,19.451228,119.451228


Вывод. Для стран RU и IN затраты на рекламу можно считать эффективными, поскольку инвестиции не только окупились, но и был получен небольшой доход. Для остальных стран реклама не была эффективной, особенно для MX, где расходы очень существенно превысили доходы.